In [1]:
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image
import imageio

In [ ]:
%config InlineBackend.figure_format = 'retina'

The reaction-diffusion system described here involves two generic chemical species U and V, whose concentration at a given point in space is referred to by variables u and v. As the term implies, they react with each other, and they diffuse through the medium. Therefore the concentration of U and V at any given location changes with time and can differ from that at other locations.

The overall behavior of the system is described by the following formula, two equations which describe three sources of increase and decrease for each of the two chemicals:


$$
\begin{array}{l}
\displaystyle \frac{\partial u}{\partial t} = D_u \Delta u - uv^2 + F(1-u) \\
\displaystyle \frac{\partial v}{\partial t} = D_v \Delta v + uv^2 - (F+k)v
\end{array}
$$

The laplacian is computed with the following numerical scheme

$$
\Delta u_{i,j} \approx u_{i,j-1} + u_{i-1,j} -4u_{i,j} + u_{i+1, j} + u_{i, j+1}
$$

The classic Euler scheme is used to integrate the time derivative.

## Initialization

$u$ is $1$ everywhere et $v$ is $0$ in the domain except in a square zone where $v = 0.25$ and $ u = 0.5$. This square located in the center of the domain is  $[0, 1]\times[0,1]$ with a size of $0.2$.


In [7]:
class GrayScottVideoGenerator:
    def __init__(self, frame_size=300, F=0.0545, k=0.062, Du=0.1, Dv=0.05):
        self.frame_size = frame_size
        self.F = F
        self.k = k
        self.Du = Du
        self.Dv = Dv
        
    def initialise(self, mask_xlim=[0.4, 0.6], mask_ylim=[0.4, 0.6]):
        n = self.frame_size
        u = np.ones((n+2,n+2))
        v = np.zeros((n+2,n+2))

        x, y = np.meshgrid(np.linspace(0, 1, n+2), np.linspace(0, 1, n+2))

        mask = (mask_xlim[0]<x) & (x<mask_xlim[1]) & (mask_ylim[0]<y) & (y<mask_ylim[1])

        u[mask] = np.random.rand(*(u[mask].shape))#0.50
        v[mask] = np.random.rand(*(v[mask].shape))#0.25

        return u, v
    
    def set_boundary_conditions(self, x):
        x[0 , :] = x[-2, :]
        x[-1, :] = x[1 , :]
        x[: , 0] = x[: ,-2]
        x[: ,-1] = x[: , 1]
        return x

    def Laplacian(self, x):
        """
        second order finite differences
        """
        return (                  x[ :-2, 1:-1] +
                 x[1:-1, :-2] - 4*x[1:-1, 1:-1] + x[1:-1, 2:] +
                              +   x[2:  , 1:-1] )
    
    def step(self, u, v):
        Lu = self.Laplacian(u)
        Lv = self.Laplacian(v)
        
        U, V = u[1:-1,1:-1], v[1:-1,1:-1]

        UVV = U*V*V
        U += self.Du*Lu - UVV + self.F*(1 - U)
        V += self.Dv*Lv + UVV - (self.F + self.k)*V

        u = self.set_boundary_conditions(u)
        v = self.set_boundary_conditions(v)
        
        return u,v
    
    def run(self, u, v, seq_len=500, save_freq=40):        
        frames = np.zeros((seq_len, *(u.shape)), dtype=np.uint8)
        for i in tqdm(range(save_freq*seq_len)):
            u,v = self.step(u,v)
            if not i % save_freq:
                frame = np.uint8(255*(v-v.min()) / (v.max()-v.min()))
                frames[int(i/save_freq)] = frame
                
        return frames
        

In [8]:
g = GrayScottVideoGenerator(frame_size=60)

In [12]:
u,v = g.initialise()#mask_xlim=[0,1], mask_ylim=[0,1])
frames = g.run(u, v, seq_len=200, save_freq=40)
imageio.mimsave('movie3.gif', frames, format='gif', fps=60)

  0%|          | 0/8000 [00:00<?, ?it/s]

In [17]:
from ipywidgets import interact, IntSlider

def display_sequence(iframe):
    
    return Image.fromarray(frames[iframe])
    
interact(display_sequence, 
         iframe=IntSlider(min=0,
                          max=len(frames)-1,
                          step=1,
                          value=0, 
                          continuous_update=True))

interactive(children=(IntSlider(value=0, description='iframe', max=199), Output()), _dom_classes=('widget-inte…

<function __main__.display_sequence(iframe)>

In [10]:
#frames_scaled = [np.uint8(255 * frame) for frame in frames]

![grayscott](movie.gif "grayscott")